In [1]:
# Date: June 11, 2025
# Author: Lydia Bullock

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

# Note - final function for imputing columns is called replace_nans. This function uses all other functions found above it.


In [1]:
import sys
print(sys.version)

3.12.2 | packaged by conda-forge | (main, Feb 16 2024, 21:00:12) [Clang 16.0.6 ]


In [2]:
#find the NaNs and determine what percentage of the data is NaN for each column of the dataframe

def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()
        
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    
    # credit for creating the table: https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction. 
    
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
    # Sort the table by percentage of missing ascending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values').round(1)

    mis_val_table_ren_columns.index.name = 'Columns'

    return mis_val_table_ren_columns


In [3]:
# Removes the columns in the data frame that do not have enough information, except for the reference column. 
#If threshold = 30, this will remove all columns that have less than 70% data (30% of the data are NaNs).

def remove_columns(df, threshold):

    percent_missing_values = missing_values_table(df)
    cols_to_drop = []

    # Remove all columns in which there are more than threshold % of NaNs (not enough data, too many NaNs)
    for col_name, percent in percent_missing_values['% of Total Values'].items():
        if percent > threshold:
            print(f"Column \"{col_name}\" exceeds the threshold.")
            cols_to_drop.append(col_name)
    
    # Drop them all at once
    df_cleaned = df.drop(columns=cols_to_drop) #the drop function does not modify the original dataframe unless inplace=True
    
    return df_cleaned
    

In [4]:
# Returns a histogram to visualize how much data is in each column

def hist_missing_values(df, threshold, save_plot=False): #threshold is percentage of missing data - if missing data is higher than threshold, bad

    df_missing_values = missing_values_table(df)
    
    #color coding the threshold
    colors = []
    for percent in df_missing_values['% of Total Values']:
        if percent > threshold:
            colors.append('red')
        else: 
            colors.append('blue')
            
    # Percentage of data that is missing
    plt.figure(figsize=(15, 6))
    plt.bar(df_missing_values.index, df_missing_values['% of Total Values'], color = colors, width=1) 
    #df_missing_values.index is the name of the column/category and df_missing_values['% of Total Values'] is the percentage of nans in each category

    # Add dotted line at threshold
    plt.axhline(y=threshold, color='black', linestyle='dotted', linewidth=1)
    
    # Labeling 
    plt.ylabel('% of Total Values')
    plt.xlabel('Category')
    plt.title('Percentage of Missing Values per Column')
    if len(df_missing_values.index) > 50:
        plt.xticks([])
    else: 
        plt.xticks(rotation=45, ha='right')

    if save_plot:
        plt.savefig('Missing_Values.png', dpi = 300)
        
    plt.tight_layout()
    plt.show()
    plt.close()
    
    # Percentage of data that exists
    plt.figure(figsize=(15, 6))
    plt.bar(df_missing_values.index, 100-df_missing_values['% of Total Values'], color = colors, width=1) 
    #df_missing_values.index is the name of the column/category and df_missing_values['% of Total Values'] is the percentage of nans in each category

    # Add dotted line at threshold
    plt.axhline(y=100-threshold, color='black', linestyle='dotted', linewidth=1)
    
    # Labeling
    plt.ylabel('% of Total Values')
    plt.xlabel('Category')
    plt.title('Percentage of Existing Data per Column')
    if len(df_missing_values.index) > 50:
        plt.xticks([])
    else: 
        plt.xticks(rotation=45, ha='right')

    if save_plot:
        plt.savefig('Existing_Data.png', dpi = 300)
        
    plt.tight_layout()
    plt.show()
    plt.close()

In [5]:
#isolate columns with more than 70% of data available (less than 30% nans)
#find the location of the nans within that column (the row number)
#find the values in the reference column that corresponds to the nans

def locate_values(df, threshold, reference_column_name):

    #dataframe of percentage of missing values
    df_missing_values = missing_values_table(df)

    # Locate and list the values corresponding to the NaNs 
    # Result is a dataframe
    
    df_combined_rows = [] #all rows of the original data frame
    relevant_rows = [] #only contains rows in which there are nans that pass the threshold % and is not the reference column
     
    for col_name, row in df_missing_values.iterrows():
        num_nans = row['Missing Values']
        percent_missing = row['% of Total Values']
        
        # Isolate columns with more than 70% of data available (less than 30% nans)
        if percent_missing <= threshold and col_name != reference_column_name:

            # Find the location of the nans within that column (the row number)
            nan_index = df[df[col_name].isna()].index

            # Find the values in the reference column that correspond to the nans
            reference_column_values = df.loc[nan_index, reference_column_name].values
            
            # Append relevant row data and combined row data: column name, NaN index, and list of values
            df_combined_rows.append({
                'Columns': col_name,
                'Row Number': nan_index.values,
                f'Values in \"{reference_column_name}\" Column Corresponding to NaNs': reference_column_values
            })
            
            relevant_rows.append({
                'Columns': col_name,
                'Row Number': nan_index.values,
                f'Values in \"{reference_column_name}\" Column Corresponding to NaNs': reference_column_values
            })
        
        elif percent_missing > threshold and col_name != reference_column_name: 
            print(f"Column \"{col_name}\" exceeds the threshold.")
            
        elif col_name == reference_column_name:
            print(f"Column \"{col_name}\" is the reference column.")
        
        else:
            print('Something is happening here. IDK.')
    
    new_info = pd.DataFrame(df_combined_rows)
    new_info = new_info.set_index('Columns')
    df_combined = pd.concat([df_missing_values, new_info], axis=1)

    result_df = pd.DataFrame(relevant_rows)
    result_df = result_df.set_index('Columns')

    return result_df, df_combined
    


In [6]:
#Locate the other values within the reference column that match the value corresponding to the NaN

def find_matching_values(df, threshold, reference_column_name):

    reference_column_values, combined = locate_values(df, threshold, reference_column_name)

    # From the table of corresponding NaN values, we must pair the index and the value together so that we can work through each pair individually.
    # First iterate through each row of the whole table to extract the index and value
    
    matching_values_rows = []
    relevant_rows = [] #only contains rows in which there are nans that pass the threshold % and is not the reference column
    
    for col_name, row in reference_column_values.iterrows():
        row_num = row['Row Number']
        corresponding_values = row[f'Values in \"{reference_column_name}\" Column Corresponding to NaNs']
        pairs = list(zip(row_num, corresponding_values))

        all_matching_values = []
        # Second, loop through each index-value pair
        for index, value in pairs:
            
            # Find the other values in the reference column that match the value we are on - below actually finds the entire row
            matching_rows = df[(df[reference_column_name] == value) & (df.index != index)]
            
            # From the matching_rows dataframe, isolate the values in the column that we are on (col_name)
            matching_values = matching_rows[col_name].values

            all_matching_values.append(matching_values)
    
        matching_values_rows.append({'Columns': col_name,
                                    f'Values in Column that Correspond to \"{reference_column_name}\" Value': all_matching_values}) 
        
            
    all_matching_values = pd.DataFrame(matching_values_rows)
    all_matching_values = all_matching_values.set_index('Columns')
    df_combined = pd.concat([combined, all_matching_values], axis=1)

    return all_matching_values, df_combined
    

In [7]:
# Calculate the mode or mean of the list of values that correspond to the NaN

def find_mode_mean(df, threshold, reference_column_name):

    matching_values, combined = find_matching_values(df, threshold, reference_column_name)

    mode_mean_rows = []
    for col_name, row in matching_values.iterrows():
        values = row[f'Values in Column that Correspond to \"{reference_column_name}\" Value']

        # Check the data type in the original column to determine if we need to use mean or mode:

        # Ignore the null values because that is what we are replacing
        column_values = df[col_name].dropna() 

        # Check if data type is binary (with special cases)
        # -55 = I don't know (par_0800)
        # -44 = I rarely/never skip lunch (diet_0600)
        if np.isin(column_values.unique(), [0, 1, -55, -44]).all():
            column_type = 'binary'    

        # Check if data type is numeric. These columns are the scale scores that we couldn't bin/categorize
        elif pd.api.types.is_numeric_dtype(column_values):
            column_type = 'numeric'

            # Check the decimal place of the entries (for rounding)
            decimal_place = get_decimal_places(column_values)

        # Categorical data type
        else:
            column_type = 'categorical'

    
        # Each array in values is a list a entries that match the entry in the reference column that corresponds to a NaN in another column
        # There can be multiple NaNs in a column and therefore multiple lists of matching values, so that is why we iterate
        
        mode_mean = []
        for array in values:
            
            # Remove NaNs
            arr_clean = [x for x in array if pd.notna(x)]

            # If the array is empty, append NaN and know that something went wrong
            if len(arr_clean) == 0:
                mode_mean.append(np.nan)
                print('The matching values array should not be empty.')

            # If binary or categorical, append the mode
            elif column_type == 'binary' or column_type == 'categorical': 
                # If two or more values occur at the same frequency, just pick the first 
                mode_series = pd.Series(arr_clean).mode()
                mode_mean.append(mode_series.iloc[0]) # mode() can return multiple modes, so pick the first one

            # If numeric, append the rounded mean
            elif column_type == 'numeric':
                mean = np.mean(arr_clean)
                rounded_mean = round_up(mean, decimal_place)
                mode_mean.append(rounded_mean)

        mode_mean_rows.append({'Columns': col_name,
                                    'Mode/Mean': mode_mean})
    
    all_modes_means = pd.DataFrame(mode_mean_rows)
    all_modes_means = all_modes_means.set_index('Columns')
    df_combined = pd.concat([combined, all_modes_means], axis=1)


    return all_modes_means, df_combined


In [8]:
# From a column, determine to which decimal place the entries are rounded

def get_decimal_places(column):
   
    decimals = []
    for val in column.dropna():
        if isinstance(val, float):
            text = str(val)
            if '.' in text:
                decimals.append(len(text.rstrip('0').split('.')[-1])) #ignores trailing zeroes
    
    return max(decimals) if decimals else 0


In [9]:
# Quick math to always round a number up

def round_up(value, decimal_places):
    
    factor = 10 ** decimal_places
    
    return math.ceil(value * factor) / factor


In [10]:
# FINAL IMPUTATION FUNCTION

def replace_nans(df, threshold, reference_column_name):

    #Get dfs with the mode/mean column included
    all_modes_means, combined = find_mode_mean(df, threshold, reference_column_name)

    # Get the dataframe of the original data (rows without ess_0900 removed and columns with over 30% NaNs removed)
    removed_columns_df_copy = df.copy()

    # Iterate through the whole table and pair up the mode/mean with the location of the original NaN
    for col_name, row in all_modes_means.iterrows():
        row_num = combined.loc[col_name]['Row Number']
        replacements = row['Mode/Mean'] 
        pairs = list(zip(row_num, replacements))

        # Replace the NaN with the mode/mean value
        for index, value in pairs:
            removed_columns_df_copy.loc[index, col_name] = value
    
    return removed_columns_df_copy


In [11]:
# Puts the total ess score into categories

def categorize_0900_scores(df, dataset_number):
    
    new_column = []
    df_copy = df.copy()

    # Iterate through the rows of the dataframe
    for index, row in df.iterrows():

        #Depending on the range the value falls in and the dataset type, assign EDS categories to a new column

        # The simple version - two categories - Normal or Sleepy
        if dataset_number == 1:
            
            if row['ess_0900'] <= 10:
                new_column.append('Normal')
                
            elif row['ess_0900'] > 10:
                new_column.append('Sleepy')
                
        # The more nuanced version - more categories
        if dataset_number == 2:
            
            if row['ess_0900'] <= 5:
                new_column.append('Lower Normal')
                
            elif row['ess_0900'] > 5 and row['ess_0900'] <= 10:
                new_column.append('Higher Normal')
                
            elif row['ess_0900'] > 10 and row['ess_0900'] <= 12:
                new_column.append('Mild')
                
            elif row['ess_0900'] > 12 and row['ess_0900'] <= 15:
                new_column.append('Moderate')

            elif row['ess_0900'] > 15:
                new_column.append('Severe')

    # Add the new column to the full df
    #ess_categories = pd.DataFrame(new_column)
    #df_combined = pd.concat([df, ess_categories], axis=1)

    col_index = df.columns.get_loc('ess_0900')
    df_copy.insert(col_index+1, 'ess_total_category', new_column)

    return df_copy
    

In [12]:
# Categorizing columns
# It will be too inefficient to try to automate this right now, so I will not write a function (code is just like categorize_0900_scores)
# I will manually change the column name, ranges, and category names to replace numbers with the strings of each category

def categorize_columns(df):
    new_column = []
    
    # Iterate through the rows of the dataframe
    for index, row in df.iterrows():
    
        # Depending on the range the values fall in, assign categories as strings to a new column
                
        if row['isi_score'] <= 7:
            new_column.append('Not Clinically Significant')
    
        elif row['isi_score'] > 7 and row['isi_score'] <= 14:
            new_column.append('Sub-Threshold Insomnia')
            
        elif row['isi_score'] > 14 and row['isi_score'] <= 21:
            new_column.append('Moderate Insomnia')
    
        elif row['isi_score'] > 11 and row['isi_score'] <= 28:
            new_column.append('Severe Insomnia')
    
        elif pd.isna(row['isi_score']):
            new_column.append(np.nan)
    
    
    # Replace the old column with the new column in the copied dataframe
    df['isi_score'] = new_column

    return df
